In [1]:
import os
from maze_env import Maze
import time

def run_maze():
    step = 0
    opt_rate = -100
    if RL.is_train:
        fout = open('./opt_hist/exp1_adam.csv', 'w')
        fout.write(str('opt_1,opt_3,opt_5\n'))
    else:
        fout = open('./opt_hist/'+env.test_folder+'.csv', 'w')
        fout.write(str('Dataset,Frame,Synthesized View,p1,p2,p3\n'))
    
    top_1 = 0
    top_3 = 0
    top_5 = 0
    done_count = 0
    for episode in tqdm(range(RL.episode)):
        # initial observation
        observation = env.reset()

        while True:
            # fresh env
            env.render()
            # RL choose action based on observation
            
            action = RL.choose_action(observation, episode)
            # RL take action and get next observation and reward
            observation_, reward, done = env.step(action)
            
            if RL.is_train:
                if done:
                    top_1 = top_1 + env.hit_top_1
                    top_3 = top_3 + env.hit_top_3
                    top_5 = top_5 + env.hit_top_5
                    done_count = done_count + 1

                RL.store_transition(observation[0], action, reward, observation_, observation[1:])

                if (step > 1000) and (step % 10 == 0):
                    is_update = RL.learn()
                    if is_update:
                        temp_top_1_rate = top_1/done_count
                        temp_top_3_rate = top_3/done_count
                        temp_top_5_rate = top_5/done_count
                        if (opt_rate <= temp_top_1_rate):
                            opt_rate = temp_top_1_rate
                            RL.save_weight(episode, './weights_exp1/')
                            print("[*%d] OPT rate %.4f, %.4f, %.4f" % (episode, temp_top_1_rate,temp_top_3_rate,temp_top_5_rate))
                        else:
                            print("[%d] OPT rate %.4f, %.4f, %.4f" % (episode, temp_top_1_rate,temp_top_3_rate,temp_top_5_rate))


                        fout.write(str('%d,%.4f,%.4f,%.4f\n' % (episode, temp_top_1_rate,temp_top_3_rate,temp_top_5_rate)))
                        fout.flush()    
                        top_1 = 0
                        top_3 = 0
                        top_5 = 0
                        done_count = 0

            # swap observation
            observation[0] = observation_

            # break while loop when end of this episode
            if done:
                if not RL.is_train:
                    fout.write("%s,%d,%d,%d,%d,%d\n" % (env.ob.r_db,env.ob.r_frame,env.ob.r_tvs,
                                                        env.cur_state[0],env.cur_state[1],env.cur_state[2]))
                break
            step += 1

    # end of game
    print('game over')
    fout.close()
    env.destroy()


In [2]:
def onKeyPress(event):
    if (event.char == "w"):
        env.take_action(0)
    elif (event.char == "d"):
        env.take_action(1)
    elif (event.char == "s"):
        env.take_action(2)
    elif (event.char == "a"):
        env.take_action(3)
    elif (event.char == "q"):
        env.destroy()

In [3]:
# if __name__ == "__main__":
env = Maze()
env.bind('<KeyPress>', onKeyPress)
env.mainloop()

you got it !!!
you die !!!
you got it !!!
you die !!!
you got it !!!
you got it !!!
you die !!!
you got it !!!
you die !!!


In [4]:
print('{0}x{1}, act_space = {2}'.format(env.MAZE_Limit[0],env.MAZE_Limit[1], len(env.action_space)))

6x8, act_space = 4
